In [7]:
from dotenv import load_dotenv
import os

import google.generativeai as genai

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

import ast
import pandas as pd
from collections import defaultdict

# === Load and preprocess the dataset ===
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    return df[df["sentiment"] == "Negative"]  # Only keep negative sentiment


# === Prepare label -> reviews mapping ===
def prepare_label_to_reviews(df, max_reviews_per_label=10):
    label_to_reviews = defaultdict(list)

    for _, row in df.iterrows():
        review = row["text"]
        labels = row["labels"]

        # Parse stringified list to actual list if needed
        if isinstance(labels, str):
            try:
                labels = ast.literal_eval(labels)
            except Exception:
                labels = []

        for label in labels:
            clean_label = label.strip().lower()
            label_to_reviews[clean_label].append(review)

    # Deduplicate and truncate to max N reviews per label
    for label in label_to_reviews:
        unique_reviews = list(dict.fromkeys(label_to_reviews[label]))
        label_to_reviews[label] = unique_reviews[:max_reviews_per_label]

    return label_to_reviews


# === Generate suggestions using Gemini ===
def generate_gemini_suggestions(label_to_reviews, model_name="gemini-1.5-flash", max_reviews=10):
    model = genai.GenerativeModel(model_name)
    suggestions = {}

    for label, reviews in label_to_reviews.items():
        if not reviews:
            continue

        selected_reviews = reviews[:max_reviews]

        prompt = f"""You are an AI assistant. Based on the following user reviews with negative sentiment about the category '{label}', generate clear, concise suggestions to improve the app in this area.

Reviews:
{chr(10).join('- ' + r for r in selected_reviews)}

Output:
- List actionable improvements
- Focus on solving the pain points mentioned in the reviews
"""

        try:
            response = model.generate_content(prompt)
            suggestions[label] = response.text.strip()
        except Exception as e:
            suggestions[label] = f"⚠️ Error generating suggestion: {e}"

    return suggestions


# === Run the pipeline ===
if __name__ == "__main__":
    # Load data and prepare mappings
    neg_df = load_data("../Dataset/gpreviews_2.csv")
    label_to_reviews = prepare_label_to_reviews(neg_df)

    # Generate improvement suggestions
    suggestions = generate_gemini_suggestions(label_to_reviews)

    # Display results
    for label, suggestion in suggestions.items():
        print(f"\n💡 Suggestions for '{label}':\n")
        print(suggestion)



💡 Suggestions for 'user experience':

## Actionable Improvements for App User Experience:

Based on the negative user reviews, here's a list of actionable improvements focused on solving the pain points:

**1. Improve Account Management & Authentication:**

* **Action:** Implement robust account recovery mechanisms.  Provide clear error messages for invalid credentials and ensure users receive notifications for account-related events (e.g., forced sign-outs).  Investigate and fix the "invalid credentials" bug.

**2. Enhance Content Discovery & Personalization:**

* **Action:**  Revise the home screen personalization. Offer users clear options to display their subscribed content *without* mandatory recommendations. Provide a setting to toggle recommendations on/off.  Consider reverting to the previous API if feasible, based on user feedback.  Investigate and fix inaccuracies in episode tracking and captions.


**3. Fix Messaging & Chat Functionality:**

* **Action:**  Debug the chat sy